# Запрос информации о количестве посещений страниц поэтов в русскоязычной Википедии

Анализируем число заходов на страницу по месяцам.

In [1]:
import pandas as pd

poets = pd.read_csv('data/poets-joined.csv')
poets.rename(columns={'Unnamed: 0': 'title'}, inplace=True)

poets

,title,pageid,ns,por_pageid,por_ns,rwp_pageid,rwp_ns
0,Проект:Литература/Списки/Русские поэты Украины,89076.0,104.0,NaN,NaN,NaN,NaN
1,Проект:Литература/Списки/Русские поэты XIX века,87816.0,104.0,NaN,NaN,NaN,NaN
2,Поэтессы Серебряного века,4692805.0,0.0,NaN,NaN,NaN,NaN
3,Русские поэты Серебряного века,87820.0,0.0,NaN,NaN,NaN,NaN
4,Русские поэты-футуристы,89092.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
3612,Яжевика,NaN,NaN,NaN,NaN,3331686.0,0.0
3613,"Якушева, Ада",NaN,NaN,NaN,NaN,79145.0,0.0
3614,"Янбулатова, Рагида Саитгалеевна",NaN,NaN,NaN,NaN,3788328.0,0.0
3615,"Яшина, Роза Ивановна",NaN,NaN,NaN,NaN,8739019.0,0.0


## Запрашиваем статистику для страниц

Описание API [здесь](https://wikitech.wikimedia.org/wiki/Analytics/AQS/Pageviews).

Данные доступны начиная в 01.08.2015.

In [8]:
import requests

S = requests.Session()

start_date = '20150801'
end_date = '20210930'

headers = {
    'User-Agent': 'RussianPoetsResearchBot/1.0 (https://github.com/agbragin/russian-poetry-wikipedia-stats; https://github.com/agbragin)'
}

def request_stats(article, start_date, end_date):
    URL = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/ru.wikipedia/all-access/all-agents/{0}/monthly/{1}/{2}'\
        .format(article, start_date, end_date)

    r = S.get(URL, headers=headers)
    data = r.json()

    if 'items' not in data:
        return pd.Series()

    return pd.Series({item['timestamp']:item['views'] for item in data['items']})

result = poets.merge(poets.apply(lambda x: request_stats(x['title'], start_date, end_date), axis=1), left_index=True, right_index=True)
result

<ipython-input-8-94c59988f2ea>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()


,title,pageid,ns,por_pageid,por_ns,rwp_pageid,rwp_ns,2015080100,2015090100,2015100100,...,2020120100,2021010100,2021020100,2021030100,2021040100,2021050100,2021060100,2021070100,2021080100,2021090100
0,Проект:Литература/Списки/Русские поэты Украины,89076.0,104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Проект:Литература/Списки/Русские поэты XIX века,87816.0,104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Поэтессы Серебряного века,4692805.0,0.0,NaN,NaN,NaN,NaN,1349.0,1841.0,2456.0,...,2877.0,3034.0,3170.0,3852.0,3223.0,2658.0,2429.0,1977.0,2076.0,2730.0
3,Русские поэты Серебряного века,87820.0,0.0,NaN,NaN,NaN,NaN,5389.0,14159.0,22557.0,...,12937.0,10555.0,10606.0,12186.0,13062.0,10484.0,8352.0,5345.0,4741.0,10943.0
4,Русские поэты-футуристы,89092.0,0.0,NaN,NaN,NaN,NaN,831.0,6138.0,8135.0,...,6405.0,4121.0,4617.0,5678.0,5091.0,3892.0,3839.0,1194.0,1104.0,3455.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3612,Яжевика,NaN,NaN,NaN,NaN,3331686.0,0.0,1512.0,1528.0,1398.0,...,308.0,322.0,341.0,335.0,624.0,361.0,545.0,741.0,452.0,396.0
3613,"Якушева, Ада",NaN,NaN,NaN,NaN,79145.0,0.0,1222.0,7037.0,4053.0,...,3099.0,2903.0,2615.0,2588.0,3322.0,2501.0,2814.0,2150.0,2327.0,2735.0
3614,"Янбулатова, Рагида Саитгалеевна",NaN,NaN,NaN,NaN,3788328.0,0.0,73.0,77.0,126.0,...,91.0,64.0,90.0,81.0,99.0,191.0,57.0,51.0,60.0,50.0
3615,"Яшина, Роза Ивановна",NaN,NaN,NaN,NaN,8739019.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,172.0,97.0,79.0,40.0,51.0,43.0,48.0


In [10]:
result.to_csv('data/poet-page-visit-stats.csv', encoding='utf8')